# Get Line of stations from Renfe data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append('..')

In [3]:
import datetime

from src.robin.scraping.renfe.entities import RenfeScraper

scraper = RenfeScraper(stations_csv_path='../data/renfe/renfe_stations.csv')

print(scraper.available_stations)

The chromedriver version (116.0.5845.96) detected in PATH at /usr/local/bin/chromedriver might not be compatible with the detected chrome version (117.0.5938.132); currently, chromedriver 117.0.5938.92 is recommended for chrome 117.*, so it is advised to delete the driver in PATH and retry


{'?': 'Estaciones de Origen', '31412': 'A Coruña', '94707': 'Abrantes', '60911': 'Alicante / Alacant', '60600': 'Albacete', '06008': 'Alcantarilla-Los Romanos', '60400': 'Alcázar de San Juan', '55020': 'Algeciras', '56312': 'Almería', '99003': 'Altet Bus', '99115': 'Aguadulce Bus', '87912': 'Aix En Provence', '99114': 'Andorra-Bus', 'ANTEQ': 'Antequera (TODAS)', '87814': 'Avignon', '10400': 'Avila', '37606': 'Badajoz', 'BARCE': 'Barcelona (TODAS)', '87078': 'Beziers', '65318': 'Benicassim', 'BILBA': 'Bilbao (TODAS)', '54400': 'Bobadilla', '11014': 'Burgos Rosa Manzano', '35400': 'Cáceres', '51405': 'Cádiz', '70600': 'Calatayud', '50417': 'Campus Rabanales', '61307': 'Cartagena', '65300': 'Castellón /Castelló', '37200': 'Ciudad Real', '50500': 'Córdoba', 'CUENC': 'Cuenca (TODAS)', '92201': 'Denia-Bus', '60905': 'Elda-Petrer', '03410': 'Elche AV/Elx AV', '94428': 'Entroncamento', '92157': 'Estepona Bus', '21010': 'Ferrol', '79309': 'Figueres', '79333': 'Figueres Bus', '04307': 'Figueres 

In [4]:
scraper.stations_df

,stop_id,stop_name,renfe_id,stop_lat,stop_lon
0,00000,Unknown,00000,0.000000,0.000000
1,31412,A Corunya,31412,43.352761,-8.409755
2,60911,Alicante/alacant,60911,38.344450,-0.495053
3,60600,Albacete-Los Llanos,60600,38.999384,-1.848450
4,60400,Alcazar de San Juan,60400,39.395628,-3.205744
...,...,...,...,...,...
91,71801,Barcelona-Sants,BARCE,41.379220,2.140624
92,13200,Bilbao-Abando Indalecio Prieto,BILBA,43.259609,-2.929150
93,66100,Cuenca,CUENC,40.067340,-2.136471
94,15410,Gijon,GIJON,43.535175,-5.698318


In [17]:
origin_id = scraper.get_renfe_station_id('60000')
destination_id = scraper.get_renfe_station_id('51003')

date = datetime.date(day=29, month=9, year=2023)
df_trips, df_stops = scraper.scrape_trips(origin_id=origin_id, destination_id=destination_id, init_date=date)

print(df_trips.head())

Date:  2023-09-29
Search url:  https://horarios.renfe.com/HIRRenfeWeb/buscar.do?O=MADRI&D=51003&AF=2023&MF=09&DF=29&SF=5&ID=s
madrid pta atocha almudena grandes
ciudad real
puertollano
villanueva de crdoba los pedroches
crdoba
sevilla santa justa
jerez de la frontera
puerto de santa mara
san fernando baha sur
cdiz
madrid pta atocha almudena grandes
ciudad real
puertollano
crdoba
sevilla santa justa
madrid pta atocha almudena grandes
crdoba
sevilla santa justa
madrid pta atocha almudena grandes
crdoba
sevilla santa justa
madrid pta atocha almudena grandes
ciudad real
puertollano
crdoba
sevilla santa justa
madrid pta atocha almudena grandes
ciudad real
puertollano
crdoba
sevilla santa justa
jerez de la frontera
puerto de santa mara
san fernando baha sur
cdiz
madrid pta atocha almudena grandes
crdoba
sevilla santa justa
madrid pta atocha almudena grandes
crdoba
sevilla santa justa
madrid pta atocha almudena grandes
crdoba
sevilla santa justa
madrid pta atocha almudena grandes
crdoba
sevil

In [12]:
df_stops.head()

,service_id,stop_id,arrival,departure
0,02074_29-09-2023-06.27,60000,0,0
1,02074_29-09-2023-06.27,37200,52,54
2,02074_29-09-2023-06.27,37300,69,71
3,02074_29-09-2023-06.27,37704,104,105
4,02074_29-09-2023-06.27,00000,303,-387


In [13]:
result_dict = {}

grouped = df_stops.groupby("service_id")

for name, group in grouped:
    sub_dict = {}
    for index, row in group.iterrows():
        sub_dict[row['stop_id']] = (row['arrival'], row['departure'])
    result_dict[name] = sub_dict

# print(result_dict)

In [14]:
trips = list(set(tuple(service.keys())for service in result_dict.values()))
# print(trips)

In [15]:
# Initialize line with max length trip
line_stations = list(trips.pop(trips.index(max(trips, key=len))))

# Complete corridor with other stops that are not in the initial defined corridor
for trip in trips:
    for i, station in enumerate(trip):
        if station not in line_stations:
            # If station is the last one, append it to the end of the corridor
            if i == len(trip) - 1:
                line_stations.append(station)
            else:
                # If station is not the last one, insert it in the corridor before the next station
                index = line_stations.index(trip[i + 1])
                line_stations.insert(index, station)

print(line_stations)

['60000', '37200', '37300', '37704', '00000', '51003', '51300', '51400', '70101']


In [16]:
mapped_names = scraper.stations_df.set_index('stop_id')['stop_name'].to_dict()
line_stations_names = list(map(mapped_names.get, line_stations))

print(line_stations_names)

['Madrid-Puerta de Atocha', 'Ciudad Real', 'Puertollano', 'Villanueva de Cordoba-Los Pedroches', 'Unknown', 'Sevilla-Santa Justa', 'Jerez de La Frontera', 'Puerto de Santa Maria', 'San Fernando Henares']
